ライブラリをインストールします。

In [ ]:
# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform google-cloud-discoveryengine google-auth --upgrade --user

ランタイムを再起動します。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

`Vertex AI User`権限のあるアカウントでログインします。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

Cloud Console で Vertex AI Search のアプリを作成してください。検索対象の文書も登録しておいてください。
プロジェクトIDとDATASTORE_IDを確認して変更してください。

In [ ]:
PROJECT_ID = "llm20230620"  # @param {type:"string"}
LOCATION = "global"
DATASTORE_ID = "es-grounding-ds_1693576321287" # @param {type:"string"}


質問定義するで。

In [ ]:
search_query = "Cloud Spannerの構成パターンとそれぞれの可用性は？"

Search モードで Summarization を、安定バージョンで。

In [ ]:
from typing import List
from google.cloud import discoveryengine_v1alpha as discoveryengine
from google.api_core.client_options import ClientOptions

client = discoveryengine.SearchServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-discoveryengine.googleapis.com")
)

# `SearchRequest` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest
request = discoveryengine.SearchRequest(
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
#    serving_config=client.serving_config_path(
##        project=PROJECT_ID,
 #       location=LOCATION,
#        data_store=DATASTORE_ID,
#        serving_config="default_search:search",
#    ),
    serving_config = f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/default_collection/dataStores/{DATASTORE_ID}/servingConfigs/default_search:answer",
   query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
    ),
    spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
        mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
    ),
    query=search_query,
    page_size=10,
    # `ContentSearchSpec` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries#summary-model
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable"
            )
        ),
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets#extractive-segments
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=4,
            max_extractive_segment_count=4,
            return_extractive_segment_score=True,
        ),
    ),
)

response = client.search(request)


サマリーの確認

In [ ]:
response.summary.summary_text

やっぱ、応答全部見たい

In [ ]:
response

Answer API で ReAct する。

https://cloud.google.com/generative-ai-app-builder/docs/answer

In [ ]:
from typing import List
from google.cloud import discoveryengine_v1alpha as discoveryengine
from google.api_core.client_options import ClientOptions

client = discoveryengine.ConversationalSearchServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-discoveryengine.googleapis.com")
)
client.serving_config_path(
        project=PROJECT_ID,
        location=LOCATION,
        data_store=DATASTORE_ID,
        serving_config="default_search:answer",
    ),
# https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.AnswerQueryRequest
request = discoveryengine.AnswerQueryRequest(
    serving_config = f"projects/{PROJECT_ID}/locations/{LOCATION}/collections/default_collection/dataStores/{DATASTORE_ID}/servingConfigs/default_search:answer",
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.AnswerQueryRequest.QueryUnderstandingSpec
    query_understanding_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec(
        query_rephraser_spec = discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryRephraserSpec(
            max_rephrase_steps = 5
        )
    ),
    answer_generation_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec(
        ignore_low_relevant_content = True,
        model_spec = discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.ModelSpec(
            model_version = "stable",
        ),
        prompt_spec = discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.PromptSpec(
              preamble = "できるだけ検索で取得したコンテキスト情報の文言を利用し、専門家でも分かるように詳細に回答を作成してください。回答には引用を示してください。"
        ),
        include_citations = True,
   ),
    query = discoveryengine.Query(
        text = search_query
    ),
#    search_spec = discoveryengine.AnswerQueryRequest.SearchSpec(
#        search_params = discoveryengine.AnswerQueryRequest.SearchSpec.SearchParams(
#            max_return_results = 3
#        ),
#    )
)

response = client.answer_query(request)


In [ ]:
response.answer.answer_text

In [ ]:
for step in response.answer.steps:
    print(step.thought)
    for action in step.actions:
        print(action.search_action.query)

In [ ]:
response

Chunksモードのデータソースに対して、chunkを取り出します。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
LOCATION = "global"
DATASTORE_ID = "<your_datastore_id>" # @param {type:"string"}

In [ ]:
# `SearchRequest` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest
request2 = discoveryengine.SearchRequest(
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config=client.serving_config_path(
        project=PROJECT_ID,
        location=LOCATION,
        data_store=DATASTORE_ID,
        serving_config="default_search:search",
    ),
    query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
    ),
    spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
        mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
    ),
    query=search_query,
    page_size=10,
    # `ContentSearchSpec` https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1alpha.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        search_result_mode="CHUNKS",#"CHUNKS"
    ),
)

response2 = client.search(request2)

In [ ]:
response2